## Import Packages

In [1]:
import zipfile as zf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import cv2
import os
from tqdm import tqdm_notebook as tqdm
from PIL import Image

import future

%load_ext autoreload
%autoreload 2

## Setup tensorboard

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/experiment_1')

## Define the ENet model

We decided to model following residual blocks as separate class to model ENET encoder and decoder:
    - Initial block
    - RDDNeck - class for regular, downsampling and dilated bottlenecks
    - ASNeck - class for asymetric bottlenecks
    - UBNeck - class for upsampling bottlenecks

ENET architecture is autoencoder based model and is divided into 5 sub-blocks. Pleas refer [ENET paper](https://arxiv.org/pdf/1606.02147.pdf) for details of each sub-block. ENET building blocks code is taken from [here](https://github.com/iArunava/ENet-Real-Time-Semantic-Segmentation).

Fast scene understanding uses first 2 sub-blocks as encoder and remaining 3 as decoder. In this implemantation, there is 1 shared encoder and 3 separate decoder for 3 tasks(instance segementation, semantic segmentation, Depth estimation )

In [17]:
from models.ENetDecoder import ENetDecoder
from models.ENetEncoder import ENetEncoder

class BranchedENet(nn.Module):
    def __init__(self, C):
        super().__init__()
        
        # Define class variables
        # C - number of classes
        self.C = C
        
        self.enc = ENetEncoder(C)
        
        self.dec1 = ENetDecoder(C)
        self.dec2 = ENetDecoder(C)
        self.dec3 = ENetDecoder(C)
        
        
    def forward(self, x):
        # Output of Encoder
        x, i1, i2 = self.enc(x)
        # output of all 3 decoder in list
        x = torch.stack([self.dec1(x, i1, i2), self.dec2(x, i1, i2), self.dec3(x, i1, i2)])
        return x

## Instantiate the ENet model

In [18]:
enet = BranchedENet(12)

In [12]:
# Checking if there is any gpu available and pass the model to gpu or cpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
enet = enet.to(device)

## Define Dataloader

In [19]:
from data.cityscapes import Cityscapes as dataset

In [20]:
height = 512
width = 1024
dataset_dir = 'data/cityscape'
image_transform = transforms.Compose(
        [transforms.Resize((height,width)),transforms.ToTensor()])
train_set = dataset(dataset_dir,transform=image_transform)

train_loader = data.DataLoader(train_set,batch_size=1,shuffle=True,
        num_workers=1)

In [21]:
dataiter = iter(train_loader)
img, label, inst, dpth = dataiter.next()

writer.add_graph(enet, img)
writer.close()

In [23]:
## Need to add in training loop
# writer.add_scalar('training loss',running_loss / 1000,epoch * len(trainloader) + i)

# for n_iter in range(100):
#     writer.add_scalar('Loss/train', np.random.random(), n_iter)
#     writer.add_scalar('Loss/test', np.random.random(), n_iter)
#     writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
#     writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

## 3 - Losses(todo)
(1) Semantic Segmentation Loss

(2) Instantance Segmentation Loss

(3) Depth Estimation Loss

In [ ]:
# def compute_instance_cost(a_C, a_G):
#     M = {}
#     tools = require 'tools/tools'

#     in_margin = 0.5
#     out_margin = 1.5
#     Lnorm = 2

#     function norm(inp, L)
#         n
#         if (L == 1) then
#             n = torch.sum(torch.abs(inp), 1)
#         else
#             n = torch.sqrt(torch.sum(torch.pow(inp, 2), 1) + 1e-8)
#         end
#         return n
#     end

#     -- prediction: batchsize x nDim x h x w
#     -- labels: batchsize x classes x h x w

#     local lossf =
#         function(prediction, labels)
#         local batchsize = prediction:size(1)
#         local c = prediction:size(2)
#         local height = prediction:size(3)
#         local width = prediction:size(4)
#         local nInstanceMaps = labels:size(2)
#         local loss = 0

#         M.loss_dist = 0
#         M.loss_var = 0

#         for b = 1, batchsize do
#             local pred = prediction[b] -- c x h x w
#             local loss_var = 0
#             local loss_dist = 0

#             for h = 1, nInstanceMaps do
#                 local label = labels[b][h]:view(1, height, width) -- 1 x h x w
#                 local means = {}
#                 local loss_v = 0
#                 local loss_d = 0

#                 -- center pull force
#                 for j = 1, label:max() do
#                     local mask = label:eq(j)
#                     local mask_sum = mask:sum()
#                     if (mask_sum > 1) then
#                         local inst = pred[mask:expandAs(pred)]:view(c, -1, 1) -- c x -1 x 1

#                         -- Calculate mean of instance
#                         local mean = torch.mean(inst, 2) -- c x 1 x 1
#                         table.insert(means, mean)

#                         -- Calculate variance of instance
#                         local var = norm((inst - mean:expandAs(inst)), 2) -- 1 x -1 x 1
#                         var = torch.cmax(var - (in_margin), 0)
#                         local not_hinged = torch.sum(torch.gt(var, 0))

#                         var = torch.pow(var, 2)
#                         var = var:view(-1)

#                         var = torch.mean(var)
#                         loss_v = loss_v + var
#                     end
#                 end

#                 loss_var = loss_var + loss_v

#                 -- center push force
#                 if (#means > 1) then
#                     for j = 1, #means do
#                         local mean_A = means[j] -- c x 1 x 1
#                         for k = j + 1, #means do
#                             local mean_B = means[k] -- c x 1 x 1
#                             local d = norm(mean_A - mean_B, Lnorm) -- 1 x 1 x 1
#                             d = torch.pow(torch.cmax(-(d - 2 * out_margin), 0), 2)
#                             loss_d = loss_d + d[1][1][1]
#                         end
#                     end

#                     loss_dist = loss_dist + loss_d / ((#means - 1) + 1e-8)
#                 end
#             end

#             loss = loss + (loss_dist + loss_var)
#         end

#         loss = loss / batchsize + torch.sum(prediction) * 0

#         return loss
#     end

# return lossf


In [ ]:
# def compute_huber_loss(input_i, label):
#     local mask = label:gt(0)
#     local d = input_i[mask] - label[mask]
#     local ds = d:size(1)

#     local da = torch.abs(d)
#     local d2 = torch.pow(d, 2)

#     local th = 1 / 5 * torch.max(da)
#     local mask2 = torch.gt(da, th)
#     da[mask2] = (d2[mask2] + (th * th)) / (2 * th)

#     return 1 / ds * torch.sum(da)

# return loss

In [ ]:
# local grad = require 'autograd'

# def lossfunction(lossf_name, weights):
#     if (lossf_name == 'softmaxLoss') then
#         lossfunction = cudnn.SpatialCrossEntropyCriterion(weights)
#     elseif (lossf_name == 'huberLoss') then
#         lossfunction = grad.nn.AutoCriterion('depthLoss_huber')(require 'lossf/myHuberLoss')
#     elseif (lossf_name == 'instanceLoss') then
#         lossfunction = grad.nn.AutoCriterion('instance_loss')(require 'lossf/myInstanceLoss')
#     else
#         assert(false, 'Cannot load lossfunction ' .. opts.lossf)
#     end

#     return lossfunction
# end

# return getLoss

# Step 5 and 6 has been done in dataloader

## 5 - Define the loader that will load the input and output images(todo)


In [ ]:
# def loader(training_path, segmented_path, batch_size, h=320, w=1000):
#     filenames_t = os.listdir(training_path)
#     total_files_t = len(filenames_t)
    
#     filenames_s = os.listdir(segmented_path)
#     total_files_s = len(filenames_s)
    
#     assert(total_files_t == total_files_s)
    
#     if str(batch_size).lower() == 'all':
#         batch_size = total_files_s
    
#     idx = 0
#     while(1):
#       # Choosing random indexes of images and labels
#         batch_idxs = np.random.randint(0, total_files_s, batch_size)
            
        
#         inputs = []
#         labels = []
        
#         for jj in batch_idxs:
#           # Reading normalized photo
#             img = plt.imread(training_path + filenames_t[jj])
#           # Resizing using nearest neighbor method
#             img = cv2.resize(img, (h, w), cv2.INTER_NEAREST)
#             inputs.append(img)
          
#           # Reading semantic image
#             img = Image.open(segmented_path + filenames_s[jj])
#             img = np.array(img)
#           # Resizing using nearest neighbor method
#             img = cv2.resize(img, (h, w), cv2.INTER_NEAREST)
#             labels.append(img)
         
#         inputs = np.stack(inputs, axis=2)
#       # Changing image format to C x H x W
#         inputs = torch.tensor(inputs).transpose(0, 2).transpose(1, 3)
        
#         labels = torch.tensor(labels)
        
#         yield inputs, labels

## 6 - Define the class weights(todo)

In [ ]:
# def get_class_weights(num_classes, c=1.02):
#     pipe = loader('images/train/', 'images/trainannot/', batch_size='all')
#     _, labels = next(pipe)
#     all_labels = labels.flatten()
#     print(all_labels)
#     each_class = np.bincount(all_labels, minlength=num_classes)
#     prospensity_score = each_class / len(all_labels)
#     class_weights = 1 / (np.log(c + prospensity_score))
#     return class_weights

# class_weights = get_class_weights(12)

## 7 - Define the Hyperparameters(todo)

In [ ]:
lr = 5e-4
batch_size = 1

criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights).to(device))
optimizer = torch.optim.Adam(enet.parameters(), 
                             lr=lr,
                             weight_decay=2e-4)

print_every = 5
eval_every = 5

## 8 - Training loop(todo)

In [ ]:
train_losses = []
eval_losses = []

bc_train = 367 // batch_size # mini_batch train
bc_eval = 101 // batch_size  # mini_batch validation

# Define pipeline objects
pipe = loader('images/train/', 'images/trainannot/', batch_size)
eval_pipe = loader('images/val/', 'images/valannot/', batch_size)

epochs = 100

In [ ]:
images, labels = pipe

In [ ]:
# Train loop

for e in range(1, epochs+1):
    
    
    train_loss = 0
    print ('-'*15,'Epoch %d' % e, '-'*15)
    
    enet.train()
    
    for _ in tqdm(range(bc_train)):
        X_batch, mask_batch = next(pipe)
        
        # assign data to cpu/gpu
        X_batch, mask_batch = X_batch.to(device), mask_batch.to(device)

        optimizer.zero_grad()
        
        out = enet(X_batch.float())

        # loss calculation for depth
        loss = criterion(out[0], mask_batch.long())
        # update weights
        loss.backward(retain_graph=True)
        #optimizer.step()

        train_loss += loss.item()
        
        # loss calculation for class instance
        loss = criterion(out[1], mask_batch.long())
        # update weights
        loss.backward(retain_graph=True)
        #optimizer.step()

        train_loss += loss.item()

        # loss calculation for class segmentation
        loss = criterion(out[2], mask_batch.long())
        # update weights
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        

    print ()
    train_losses.append(train_loss)
    
    if (e+1) % print_every == 0:
        print ('Epoch {}/{}...'.format(e, epochs),
                'Loss {:6f}'.format(train_loss))
    
    if e % eval_every == 0:
        with torch.no_grad():
            enet.eval()
            
            eval_loss = 0

            # Validation loop
            for _ in tqdm(range(bc_eval)):
                inputs, labels = next(eval_pipe)

                
                inputs, labels = inputs.to(device), labels.to(device)
                    
                
                out = enet(inputs)
                
                out = out.data.max(1)[1]
                
                eval_loss += (labels.long() - out.long()).sum()
                
            
            print ()
            print ('Loss {:6f}'.format(eval_loss))
            
            eval_losses.append(eval_loss)
        
    if e % print_every == 0:
        checkpoint = {
            'epochs' : e,
            'state_dict' : enet.state_dict()
        }
        torch.save(checkpoint, '/content/ckpt-enet-{}-{}.pth'.format(e, train_loss))
        print ('Model saved!')

print ('Epoch {}/{}...'.format(e, epochs),
       'Total Mean Loss: {:6f}'.format(sum(train_losses) / epochs))

In [ ]:
import os

In [ ]:
for path, _, files in os.walk(folder):